## Flight Features

LAgent is able to： 
1. show the historical average monthly flight price to help user decide the time for the LA trip based on the departure city
2. show the historical and estimated on-time rate based on user's preference of airline, so that the user can estimate their on-time rate

Specifically, LAgent can:
Feature 1.0: ask users for the departure city 
Feature 1.1: show the average monthly airline price from the departure city the user selected to LAX airport by table and visulization
Feature 2.0: ask users for the airline he/she would like to talke
Feature 2.1: show the historical and predicted flight on-time rate based on time series algorithm 
     

In [6]:
# import necessary package
import bs4
import requests
import numpy as np
import pandas as pd

#### Function 1: Data preparation

In [7]:
# the web show the airfare history from Detroit to Los Angeles
url = "https://www.faredetective.com/farehistory/flights-from-Detroit-DTW-to-Los_Angeles-LAX.html"

In [8]:
# set up requests to grab content form the url, and set it as a Beautiful Soup object.
try:
    response = requests.get(url)
    response.raise_for_status()
except:
    print('exception')

soup = bs4.BeautifulSoup(response.content, 'html.parser')

In [132]:
# use Beautiful Soup to search for the city name and the corresponding links
city_selector = soup.select('table[class="table table-history"]')
cities = city_selector[0].select('a')

# Get the dictionary for city name and its corresponding web page for airfare information
city_links = []
city_name = []
for city in cities:
    city_links.append(city.get('href'))
    long_name = city.get('href').split('/')[4]
    name = long_name.split('-')[2]
    city_name.append(name)
    
# show the result
city_web = dict(zip(city_name, city_links))
cities = sorted(list(city_web.keys()))

#### Function 1: Combine the data

In [101]:
# download the city csv files and store them in the Data file
# define a function to read each files and change the index and column name
# for files which has multiple record for one month, calculate the mean of the airfare; 
# for example, Atlanta has three records for Oct 2017, which will tranform to one record by calculating the average. 
def table_transformation(i):
    df = pd.read_csv(files[i])
    df['month'] = df['year'].str[:3]
    df['year'] = df['year'].str[-4:]
    df.set_index(['year','month'], inplace = True)
    new_column = files[i][5: -4]
    df.rename(columns = {'price' : new_column}, inplace = True )
    df = df.mean(level= [0,1])
    return df

In [157]:
import glob
files = glob.glob('Data/*.csv')
#print(files)

airfare = table_transformation(0)
no_data = []

for i in list(range(1, len(files))):
    try:
        new_df = table_transformation(i)
        airfare = airfare.add(new_df, fill_value=0)
    except:
        no_data.append(files[i])
        
recent_airfare = airfare.loc[['2017', '2018']]


#### Function 1: show the historical average monthly flight price 

In [ ]:
# get user input for the departure city 
print("***********************************")
print("Please choose your departure city: ")
for index, city in enumerate(cities):
        print(index + 1, city)

inp = input("Please return the index (1 to 57) for your departure city.")
while (int(inp.strip()) >= 58 or int(inp.strip()) <= 0 ):
    inp = input("Invalid input. Please return the index (1 to 57) for your departure city.")

# get the airfare from 2017 to 2018 based on departure city given by users
user_airfare = recent_airfare[cities[int(inp.strip())- 1]]
print(user_airfare)
